In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn

In [8]:
df = pd.read_excel('D:\\BigData\\project\\High_school\\data\\2020-2019.xlsx')
df.head(2)

,학교명,등수,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시)
0,외대부고,1,69,39,30,73,36,37
1,하나고,2,62,5,57,51,2,49


In [9]:
df.학교명 = df.학교명.apply(lambda x : x+'등학교')

### 주소 / 위도경도 추출

In [10]:
# 위도경도 => url값에 @다음으로 
# 시군 => 크롤링
# 자료수정 : 고등학교로 통일

In [11]:
df.학교명

0      외대부고등학교
1       하나고등학교
2      대원외고등학교
3       상산고등학교
4      한영외고등학교
        ...   
101     양재고등학교
102    조대여고등학교
103     풍덕고등학교
104     한영고등학교
105    해운대고등학교
Name: 학교명, Length: 106, dtype: object

#### 학교 좌표 추출

In [12]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [18]:
url = 'https://www.google.co.kr/maps'
driver = webdriver.Chrome('D:\\BigData\\chromedriver.exe')

In [14]:
driver.get(url)  # google 지도 검색창 
driver.find_element_by_xpath('//*[@id="searchboxinput"]').click()  # 검색창 클릭
driver.find_element_by_xpath('//*[@id="searchboxinput"]').send_keys(df.학교명[19])  # 학교(예시) 입력
driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]').click() # 검색

time.sleep(5)
xy = driver.current_url   # url 추출

In [12]:
xy.split('@')[1].split('z/')[0].split(',')[:2][0]  # 좌표추출[위도, 경도]

'37.3477176'

In [13]:
guso = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[8]/button/div[1]/div[2]/div[1]').text

In [14]:
guso  # 도로명

'경기도 의왕시 고천동 고산로105번길 30'

##### 학교별 주소 및 좌표 col추가

In [12]:
xy.split('@')[1].split('z/')[0].split(',')

['37.3477176', '126.9575231', '17']

In [17]:
driver = webdriver.Chrome('D:\\BigData\\chromedriver.exe')

l1 = []   # 위도 모음
l2 = []   # 경도
l3 = []   # 주소

for i in df.학교명[:5] :
    url = 'https://www.google.co.kr/maps'   
    driver.get(url)  # google 지도 검색창 
    driver.find_element_by_xpath('//*[@id="searchboxinput"]').click()  # 검색창 클릭
    driver.find_element_by_xpath('//*[@id="searchboxinput"]').send_keys(i)  # 학교 입력(i == 학교명)
    driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]').click() # 검색
    time.sleep(5)
    try :
        xy = driver.current_url   # url 추출
        xy.split('@')[1].split('z/')[0].split(',')[:2]  # 좌표추출[위도, 경도]
        guso = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[8]/button/div[1]/div[2]/div[1]').text # 도로명 주소
        l1.append(xy.split('@')[1].split('z/')[0].split(',')[0]) # 위도 리스트 생성
        l2.append(xy.split('@')[1].split('z/')[0].split(',')[1]) # 경도 리스트 생성
        l3.append(guso)   # 주소 리스트 생성
    except :
        print(i)
        l1.append(np.nan)
        l2.append(np.nan)
        l3.append(np.nan)

In [17]:
df.학교명[19]

'경기외고등학교'

In [19]:
df1 = df.copy()

In [20]:
df1.위도

NameError: name 'df1' is not defined

In [21]:
df1['위도'] = l1
df1['경도'] = l2
df1['주소'] = l3

df1

NameError: name 'df1' is not defined

In [21]:
del df1['등수']

In [22]:
df1

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),위도,경도,주소
0,외대부고등학교,69,39,30,73,36,37,37.3301408,127.2538162,경기도 용인시 처인구 모현면 왕산리 232
1,하나고등학교,62,5,57,51,2,49,37.641704,126.9350002,서울특별시 은평구 진관동 연서로 535
2,대원외고등학교,58,23,35,53,20,33,37.5636956,127.0869381,서울특별시 광진구 중곡4동 용마산로22길 26
3,상산고등학교,37,26,11,38,25,13,35.8044177,127.1165053,전라북도 전주시 완산구 효자동1가 거마평로 130
4,한영외고등학교,28,7,21,23,8,15,37.5480681,127.1550363,서울특별시 강동구 상일동 동남로 832
...,...,...,...,...,...,...,...,...,...,...
101,양재고등학교,5,0,5,비공개,비공개,비공개,37.4830119,127.029857,서울특별시 서초구 서초2동 남부순환로346길 9
102,조대여고등학교,5,0,5,비공개,비공개,비공개,NaN,NaN,NaN
103,풍덕고등학교,5,1,4,비공개,비공개,비공개,37.3167426,127.087005,경기도 용인시 수지구 풍덕천동 1120-2
104,한영고등학교,5,0,5,14,0,14,37.5488717,127.1561011,서울특별시 강동구 상일동 동남로 832


In [51]:
df1[df1.학교명 == '정화여고등학교']

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),위도,경도,주소
51,정화여고등학교,9,6,3,3,2,1,NaN,NaN,NaN


In [66]:
df3 = df1[df1.위도.isnull()]

In [36]:
l1= []
l2= []
l3= []

In [64]:
url = 'https://www.google.co.kr/maps'   
driver.get(url)  # google 지도 검색창 
driver.find_element_by_xpath('//*[@id="searchboxinput"]').click()  # 검색창 클릭
driver.find_element_by_xpath('//*[@id="searchboxinput"]').send_keys('조대여고등학교')  # 학교이름 입력
driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]').click() # 검색
time.sleep(5)

In [65]:
xy = driver.current_url   # url 추출
xy.split('@')[1].split('z/')[0].split(',')[:2]  # 좌표추출[위도, 경도]
guso = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[8]/button/div[1]/div[2]/div[1]').text # 도로명 주소

l1.append(xy.split('@')[1].split('z/')[0].split(',')[0]) # 위도 리스트 생성
l2.append(xy.split('@')[1].split('z/')[0].split(',')[1]) # 경도 리스트 생성
l3.append(guso)   # 주소 리스트 생성

In [69]:
df3['위도'] = l1
df3['경도'] = l2
df3['주소'] = l3

<ipython-input-69-ff013330d69e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['위도'] = l1
<ipython-input-69-ff013330d69e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['경도'] = l2
<ipython-input-69-ff013330d69e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [103]:
df2 = df1.drop([18, 21, 37, 39, 43, 51, 54, 68, 70, 77, 92, 102],axis = 0)

In [96]:
df3.index

Int64Index([18, 21, 37, 39, 43, 51, 54, 68, 70, 77, 92, 102], dtype='int64')

In [70]:
df3

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),위도,경도,주소
18,한일고등학교,17,9,8,18,7,11,36.2178016,126.8461473,충청남도 공주시 정안면 광정리 산21-2
21,중산고등학교,16,10,6,12,7,5,37.5910122,126.6510358,서울특별시 강남구 일원로14길 22
37,용산고등학교,12,6,6,7,1,6,37.5457891,126.9774366,서울특별시 후암동
39,부산외고등학교,11,4,7,7,0,7,35.1981521,129.0692256,부산광역시 연제구 연산동 고분로 144
43,고려고등학교,10,3,7,10,5,5,35.1935611,125.7760949,광주광역시 북구 삼각동 설죽로 369
51,정화여고등학교,9,6,3,3,2,1,37.4927143,126.9756841,서울특별시 송파구 잠실동 올림픽로4길 16
54,대전동신과고등학교,8,0,8,14,0,14,36.3155522,127.4026959,대전광역시 중구 문화동 361-1
68,대진고등학교,7,5,2,11,7,4,37.492652,126.9054689,서울특별시 노원구 하계동 공릉로62가길 47
70,부산국제외고등학교,7,1,6,4,0,4,35.166213,129.0146347,부산광역시 해운대구 우동 해운대로469번길 50-1
77,이대부고등학교,7,2,5,5,1,4,37.5661394,126.9451367,서울특별시 서대문구 대신동 성산로 560


In [111]:
df4

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),위도,경도,주소
0,외대부고등학교,69,39,30,73,36,37,37.3301408,127.2538162,경기도 용인시 처인구 모현면 왕산리 232
1,하나고등학교,62,5,57,51,2,49,37.641704,126.9350002,서울특별시 은평구 진관동 연서로 535
2,대원외고등학교,58,23,35,53,20,33,37.5636956,127.0869381,서울특별시 광진구 중곡4동 용마산로22길 26
3,상산고등학교,37,26,11,38,25,13,35.8044177,127.1165053,전라북도 전주시 완산구 효자동1가 거마평로 130
4,한영외고등학교,28,7,21,23,8,15,37.5480681,127.1550363,서울특별시 강동구 상일동 동남로 832
...,...,...,...,...,...,...,...,...,...,...
101,대진고등학교,7,5,2,11,7,4,37.492652,126.9054689,서울특별시 노원구 하계동 공릉로62가길 47
102,부산국제외고등학교,7,1,6,4,0,4,35.166213,129.0146347,부산광역시 해운대구 우동 해운대로469번길 50-1
103,이대부고등학교,7,2,5,5,1,4,37.5661394,126.9451367,서울특별시 서대문구 대신동 성산로 560
104,광주숭일고등학교,5,0,5,5,0,5,35.2180995,126.8942926,광주광역시 북구 일곡동 46-24


In [114]:
df11 = pd.read_excel('./data/서울대학교 2013-2020.xlsx')
df11

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),2018(총합),2018(정시),2018(수시),...,2016(수시),2015(총합),2015(정시),2015(수시),2014(총합),2014(정시),2014(수시),2013(총합),2013(정시),2013(수시)
0,외대부고,69,39,30,73,36,37,55,24,31,...,44.0,61,31.0,30.0,92,34.0,58.0,45.0,20,25
1,하나고,62,5,57,51,2,49,55,3,52,...,53.0,54,8.0,46.0,60,8.0,52.0,44.0,2,42
2,대원외고,58,23,35,53,20,33,50,14,36,...,41.0,79,31.0,48.0,95,32.0,63.0,83.0,44,39
3,상산고,37,26,11,38,25,13,30,21,9,...,10.0,53,38.0,15.0,53,28.0,25.0,52.0,28,24
4,한영외고,28,7,21,23,8,15,34,9,25,...,23.0,31,9.0,22.0,35,10.0,25.0,19.0,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,양재고,5,0,5,비공개,비공개,비공개,4,0,4,...,9.0,10,2.0,8.0,4,1.0,3.0,7.0,2,5
102,조대여고,5,0,5,비공개,비공개,비공개,0,0,0,...,1.0,3,1.0,2.0,1,1.0,0.0,3.0,0,3
103,풍덕고,5,1,4,비공개,비공개,비공개,7,4,3,...,0.0,6,4.0,2.0,2,0.0,2.0,2.0,1,1
104,한영고,5,0,5,14,0,14,13,2,11,...,8.0,9,1.0,8.0,10,1.0,9.0,9.0,1,8


In [115]:
df11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   학교명       106 non-null    object 
 1   2020(총합)  106 non-null    int64  
 2   2020(정시)  106 non-null    int64  
 3   2020(수시)  106 non-null    int64  
 4   2019(총합)  106 non-null    object 
 5   2019(정시)  106 non-null    object 
 6   2019(수시)  106 non-null    object 
 7   2018(총합)  106 non-null    int64  
 8   2018(정시)  106 non-null    int64  
 9   2018(수시)  106 non-null    int64  
 10  2017(총합)  105 non-null    float64
 11  2017(정시)  105 non-null    float64
 12  2017(수시)  105 non-null    float64
 13  2016(총합)  106 non-null    int64  
 14  2016(정시)  103 non-null    float64
 15  2016(수시)  103 non-null    float64
 16  2015(총합)  106 non-null    int64  
 17  2015(정시)  102 non-null    float64
 18  2015(수시)  102 non-null    float64
 19  2014(총합)  106 non-null    int64  
 20  2014(정시)  100 non-null    float6

In [124]:
df12 = pd.concat([df11,df4[['위도','경도','주소']]],axis =1)  # 고등학교 목록 / 경도 위도 주소 추가

In [22]:
df12

NameError: name 'df12' is not defined

In [125]:
df12.to_excel('./data/서울대출신고등학교수정.xlsx')

In [253]:
df13 = pd.read_excel('data/서울대출신고등학교수정.xlsx')

In [254]:
df13 = df13.iloc[:,1:]
df13

,학교명,2020(총합),2020(정시),2020(수시),2019(총합),2019(정시),2019(수시),2018(총합),2018(정시),2018(수시),...,2015(수시),2014(총합),2014(정시),2014(수시),2013(총합),2013(정시),2013(수시),위도,경도,주소
0,외대부고,69,39,30,73.0,36.0,37.0,55,24,31,...,30.0,92,34.0,58.0,45.0,20.0,25.0,37.330141,127.253816,경기도 용인시 처인구 모현면 왕산리 232
1,하나고,62,5,57,51.0,2.0,49.0,55,3,52,...,46.0,60,8.0,52.0,44.0,2.0,42.0,37.641704,126.935000,서울특별시 은평구 진관동 연서로 535
2,대원외고,58,23,35,53.0,20.0,33.0,50,14,36,...,48.0,95,32.0,63.0,83.0,44.0,39.0,37.563696,127.086938,서울특별시 광진구 중곡4동 용마산로22길 26
3,상산고,37,26,11,38.0,25.0,13.0,30,21,9,...,15.0,53,28.0,25.0,52.0,28.0,24.0,35.804418,127.116505,전라북도 전주시 완산구 효자동1가 거마평로 130
4,한영외고,28,7,21,23.0,8.0,15.0,34,9,25,...,22.0,35,10.0,25.0,19.0,4.0,15.0,37.548068,127.155036,서울특별시 강동구 상일동 동남로 832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,양재고,5,0,5,NaN,NaN,NaN,4,0,4,...,8.0,4,1.0,3.0,7.0,2.0,5.0,37.492652,126.905469,서울특별시 노원구 하계동 공릉로62가길 47
102,조대여고,5,0,5,NaN,NaN,NaN,0,0,0,...,2.0,1,1.0,0.0,3.0,0.0,3.0,35.166213,129.014635,부산광역시 해운대구 우동 해운대로469번길 50-1
103,풍덕고,5,1,4,NaN,NaN,NaN,7,4,3,...,2.0,2,0.0,2.0,2.0,1.0,1.0,37.566139,126.945137,서울특별시 서대문구 대신동 성산로 560
104,한영고,5,0,5,14.0,0.0,14.0,13,2,11,...,8.0,10,1.0,9.0,9.0,1.0,8.0,35.218100,126.894293,광주광역시 북구 일곡동 46-24


In [258]:
df13 = df13.fillna(0)

In [260]:
df13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   학교명       106 non-null    object 
 1   2020(총합)  106 non-null    int64  
 2   2020(정시)  106 non-null    int64  
 3   2020(수시)  106 non-null    int64  
 4   2019(총합)  106 non-null    float64
 5   2019(정시)  106 non-null    float64
 6   2019(수시)  106 non-null    float64
 7   2018(총합)  106 non-null    int64  
 8   2018(정시)  106 non-null    int64  
 9   2018(수시)  106 non-null    int64  
 10  2017(총합)  106 non-null    float64
 11  2017(정시)  106 non-null    float64
 12  2017(수시)  106 non-null    float64
 13  2016(총합)  106 non-null    int64  
 14  2016(정시)  106 non-null    float64
 15  2016(수시)  106 non-null    float64
 16  2015(총합)  106 non-null    int64  
 17  2015(정시)  106 non-null    float64
 18  2015(수시)  106 non-null    float64
 19  2014(총합)  106 non-null    int64  
 20  2014(정시)  106 non-null    float6

In [24]:
df_dic1 = {}
for i in range(2013,2021) :
    sub = str(i)+'(총합)'
    sub1 = str(i)+'(수시)'
    sub2 = str(i)+'(정시)'
    df14 = df13[['학교명',sub,sub1,sub2,'위도','경도','주소']].sort_values(sub,ascending = False)[:50]
    df_dic1[i] = df14

NameError: name 'df13' is not defined

In [23]:
df_dic1[2020]

NameError: name 'df_dic1' is not defined

In [268]:
for i in df_dic1.keys() :
    na = str(i)+'상위50'
    df_dic1[i].to_excel('./data/'+na+'.xlsx')